# Converting raw stability data to BinaryMap object

In this notebook, I analyze the raw data reported in Nisthal et al. (2019) - *Protein stability engineering insights revealed by domain-wide comprehensive mutagenesis*. This paper constructs and purifies every single GB1 mutant. Mutant stability is measured by generating a protein unfolding curve in response to a gradient of guanidium chloride, over 2-6 biological replicates. The authors determine the concentration of denaturant at the midpoint of the unfolding transition (Cm) and the slope of the curve. ddG is then calculated as:

    ddG = mean * (Cm mutant - Cm WT)

With this equation, stabilizing mutations have **positive** ddG values, and destabilizing mutations have **negative** values. 

This data is reported in ProtaBank under [the ID gwoS2haU3](https://www.protabank.org/study_analysis/3xESLyS9/). The csv file, downloaded from 'Study Data', reports values from numerous assays. For our purposes, I'll be pulling out values reported as 'ddG(mAvg)_mean' and 'SD of ddg(mAvg)_mean', which I assume to be data aggregated over their 2-6 biological replicates. This data was reported to be more precise than ddG(H2O), the stability data calculated using dG(H2O) measurements.

Here, I read in the raw data, isolate the data of interest, and format the dataframe such that it can be turned into a BinaryMap object for downstream epistasis modeling. BinaryMap is part of the dms_variants package developed by the Bloom Lab, and more information can be found [here](https://jbloomlab.github.io/dms_variants/dms_variants.binarymap.html). The output of this notebook is a BinaryMap object describing stability of GB1 single mutants.

In [2]:
import pandas as pd
import numpy as np

import dms_variants.binarymap
import dms_variants.globalepistasis

In [3]:
raw_stability = pd.read_csv('../data/nisthal_gb1_stability.csv')
raw_stability.head()

,Sequence,Description,Ligand,Data,Units,Assay/Protocol
0,ATYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,M01A,NaN,NaN,kcal/mol,ddG(deepseq)_Olson
1,ATYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,M01A,NaN,NaN,kcal/mol,ddG_lit_fromOlson
2,ATYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,M01A,NaN,-1.777,kcal/mol·M,m-value
3,ATYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,M01A,NaN,-0.635,kcal/mol,FullMin
4,ATYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYD...,M01A,NaN,-0.510,kcal/mol,Rosetta SomeMin_ddG


In [4]:
raw_stability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18861 entries, 0 to 18860
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Sequence        18861 non-null  object 
 1   Description     18861 non-null  object 
 2   Ligand          0 non-null      float64
 3   Data            16785 non-null  float64
 4   Units           18861 non-null  object 
 5   Assay/Protocol  18861 non-null  object 
dtypes: float64(2), object(4)
memory usage: 884.2+ KB


Drop redundant columns and pull out assay data of interest

In [5]:
cols = [0,2,4]
stability_edit = raw_stability.drop(raw_stability.columns[cols], axis=1)
stability_edit.rename(columns={'Description':'aa_substitutions', 
                              'Data':'stability_score', 
                              'Assay/Protocol':'assay'}, inplace=True)

assays = ('ddG(mAvg)_mean', 'SD of ddG(mAvg)_mean')
stability_edit = stability_edit.loc[stability_edit['assay'].isin(assays)]

stability_edit.head()

,aa_substitutions,stability_score,assay
10,M01A,0.14070,ddG(mAvg)_mean
11,M01A,0.17250,SD of ddG(mAvg)_mean
24,M01D,0.19470,SD of ddG(mAvg)_mean
29,M01D,0.37950,ddG(mAvg)_mean
40,M01E,0.06883,SD of ddG(mAvg)_mean


Pivot to get columns for stability and standard deviation

In [6]:
gb1_stability = stability_edit.pivot(index='aa_substitutions',columns='assay').reset_index()
gb1_stability.columns = ['aa_substitutions', 'stability_score_sd', 'stability_score']

# add n_aa_substitutions column
gb1_stability['n_aa_substitutions'] = 1

Remove leading 0's from aa_substitutions at single digit positions, to align with BinaryMap object formatting.

In [7]:
import re

single_digit = re.compile('([A-Z])(0)(\d)([A-Z])')

gb1_stability['aa_substitutions'] = gb1_stability['aa_substitutions'].replace(
    {single_digit: r'\1\3\4'}, regex=True)

In [8]:
print(gb1_stability[300:330])

    aa_substitutions  stability_score_sd  stability_score  n_aa_substitutions
300             F52Q            0.000000         -4.00000                   1
301             F52R            0.000000         -4.00000                   1
302             F52S            0.000000         -4.00000                   1
303             F52T            0.000000         -4.00000                   1
304             F52V            0.339900         -2.99200                   1
305             F52Y            0.052500         -0.34310                   1
306              G9A            0.047960         -0.25260                   1
307              G9D            0.105800         -2.50200                   1
308              G9E            0.139100         -1.99700                   1
309              G9F            0.163400          0.00246                   1
310              G9H            0.107200         -1.22000                   1
311              G9I            0.124400         -0.59260       

In [9]:
gb1_stability.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 935 entries, 0 to 934
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   aa_substitutions    935 non-null    object 
 1   stability_score_sd  935 non-null    float64
 2   stability_score     935 non-null    float64
 3   n_aa_substitutions  935 non-null    int64  
dtypes: float64(2), int64(1), object(1)
memory usage: 29.3+ KB


Export as pickle

In [10]:
# for setting up expand=True in BinaryMap conversion
wtseq = "MQYKLILNGKTLKGETTTEAVDAATAEKVFKQYANDNGVDGEWTYDDATKTFTVTE"

# export fitness data and wtseq
import pickle
with open("gb1_stability.pkl", "wb") as f:
      pickle.dump([gb1_stability, wtseq], f)